In [1]:
import os

In [2]:
%pwd

'f:\\Anaconda Projects\\end to end wine quality fifth\\end-to-end-wine-quality-fifth\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'f:\\Anaconda Projects\\end to end wine quality fifth\\end-to-end-wine-quality-fifth'

In [5]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories


In [9]:
class ConfigurationManager:
    def __init__(
            self,
            config_filename = CONFIG_FILE_PATH,
            params_filename = PARAMS_FILE_PATH,
            schema_filename = SCHEME_FILE_PATH
    ):
        self.config = read_yaml(config_filename)
        self.params = read_yaml(params_filename)
        self.schema = read_yaml(schema_filename)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [10]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [14]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f'{filename} download with following info : \n{headers}')

        else:
            logger.info(f"File already exist of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_file:
            zip_file.extractall(unzip_path)

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-18 19:39:10,509: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-18 19:39:10,545: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-18 19:39:10,548: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-18 19:39:10,551: INFO: common: Created directory at artifacts]
[2024-01-18 19:39:10,556: INFO: common: Created directory at artifacts/data_ingestion]


[2024-01-18 19:39:15,828: INFO: 2987133021: artifacts/data_ingestion/data.zip download with following info : 
Connection: close
Content-Length: 26148
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "a0969ead3127874d2cb8be7c45ceed0b7b5aaacebd0de95291b9e1c7492c2e7e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: DC64:45E6:1219DEF:156EF70:65A9310A
Accept-Ranges: bytes
Date: Thu, 18 Jan 2024 14:09:15 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10224-MAA
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1705586955.234558,VS0,VE527
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 2b3bd03e2e7a9edf8b60bd0c361e94cfd7c3e285
Expires: Thu, 18 Jan 2024 14:14:15 GMT
Source-Age: 0

]
